In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
import networkx as nx

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]  # Get the first graph object

# Initialize model, optimizer, and loss function
model = GCN(in_channels=dataset.num_node_features, hidden_channels=16, out_channels=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients
    out = model(data.x, data.edge_index)  # Forward pass
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights
    return loss.item()

# Evaluation function
def test():
    model.eval()
    with torch.no_grad():  # No gradients needed for evaluation
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)  # Get the predicted classes
        test_accuracy = (pred[data.test_mask] == data.y[data.test_mask]).sum() / data.test_mask.sum()
        return test_accuracy.item()

# Run the training process
for epoch in range(200):  # Number of epochs
    loss = train()
    if epoch % 10 == 0:
        test_acc = test()
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')

print("Training completed.")

# Function to plot the interactive graph
def plot_interactive_graph():
    # Create a NetworkX graph from edge index
    edge_index = data.edge_index.cpu().numpy()  # Convert to NumPy array
    G = nx.Graph()

    # Add edges to the graph
    for i in range(edge_index.shape[1]):
        G.add_edge(edge_index[0, i], edge_index[1, i])

    # Node labels and colors
    labels = {i: data.y[i].item() for i in range(data.num_nodes)}
    colors = np.array([data.y[i].item() for i in range(data.num_nodes)])

    # Create position layout for nodes
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # Prepare data for Plotly
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    node_x = []
    node_y = []
    for i in range(data.num_nodes):
        x, y = pos[i]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Blues',
            size=10,
            color=colors,
            colorbar=dict(thickness=15, title='Node Class'),
            line=dict(width=2)
        ),
        text=[f'Node {i}: {labels[i]}' for i in range(data.num_nodes)]
    )

    # Create the figure
    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=dict(text='Interactive Graph Visualization of the Cora Dataset', font=dict(size=16)),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                ))


    pyo.plot(fig)  # Use pyo.iplot(fig) if you're in Jupyter Notebook

# Plot the interactive graph
plot_interactive_graph()



Epoch: 0, Loss: 1.9468, Test Accuracy: 0.4600
Epoch: 10, Loss: 0.5937, Test Accuracy: 0.7950
Epoch: 20, Loss: 0.0903, Test Accuracy: 0.7900
Epoch: 30, Loss: 0.0183, Test Accuracy: 0.7920
Epoch: 40, Loss: 0.0068, Test Accuracy: 0.7880
Epoch: 50, Loss: 0.0039, Test Accuracy: 0.7910
Epoch: 60, Loss: 0.0028, Test Accuracy: 0.7910
Epoch: 70, Loss: 0.0023, Test Accuracy: 0.7910
Epoch: 80, Loss: 0.0020, Test Accuracy: 0.7900
Epoch: 90, Loss: 0.0018, Test Accuracy: 0.7910
Epoch: 100, Loss: 0.0016, Test Accuracy: 0.7920
Epoch: 110, Loss: 0.0014, Test Accuracy: 0.7910
Epoch: 120, Loss: 0.0013, Test Accuracy: 0.7910
Epoch: 130, Loss: 0.0012, Test Accuracy: 0.7910
Epoch: 140, Loss: 0.0011, Test Accuracy: 0.7890
Epoch: 150, Loss: 0.0010, Test Accuracy: 0.7890
Epoch: 160, Loss: 0.0009, Test Accuracy: 0.7870
Epoch: 170, Loss: 0.0009, Test Accuracy: 0.7860
Epoch: 180, Loss: 0.0008, Test Accuracy: 0.7860
Epoch: 190, Loss: 0.0007, Test Accuracy: 0.7860
Training completed.
